<a href="https://colab.research.google.com/github/dagdo03/stock-return-analysis/blob/main/stockreturn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import files
import plotly.graph_objects as go
from datetime import datetime
import numpy as np
from matplotlib import pyplot

In [ ]:
df = pd.read_csv('PTBA (4).csv')
columns = ['date', 'open', 'high', 'low', 'close', 'volume']
df = df[columns]
df = df[df['volume']!=0]
df.reset_index(drop=True, inplace=True)
df.isna().sum()
df.head(10)

,date,open,high,low,close,volume
0,2002-12-23,38.850140,38.850140,37.231384,38.850140,74795000
1,2002-12-27,38.850140,38.850140,37.231384,38.850140,36812500
2,2003-01-02,38.850132,38.850132,35.612621,37.231377,149657500
3,2003-01-03,37.231377,37.231377,35.612621,37.231377,16197500
4,2003-01-06,35.612625,37.231381,33.993870,33.993870,119477500
5,2003-01-07,35.612617,35.612617,33.993862,35.612617,17337500
6,2003-01-08,33.993862,35.612617,33.993862,35.612617,2952500
7,2003-01-09,35.612625,35.612625,33.993870,33.993870,1625000
8,2003-01-10,33.993862,35.612617,33.993862,35.612617,212500
9,2003-01-13,35.612617,35.612617,33.993862,35.612617,6285000


In [ ]:
dfpl = df[4800:5048]
fig = go.Figure(data = [go.Candlestick(x=dfpl.index,
                                       open = dfpl['open'],
                                       high = dfpl['high'],
                                       low = dfpl['low'],
                                       close = dfpl['close'])])
fig.show()

In [ ]:
backcandles = 200
wind = 5
candleid = 4890

maxim = np.array([])
minim = np.array([])
minid = np.array([])
maxid = np.array([])

for i in range(candleid - backcandles, candleid + 1, wind):
  minim = np.append(minim, df.low.iloc[i:i+wind].min())
  minid = np.append(minid, df.low.iloc[i:i+wind].idxmin())
for i in range(candleid - backcandles, candleid + 1, wind):
  maxim = np.append(maxim, df.low.iloc[i:i+wind].max())
  maxid = np.append(maxid, df.low.iloc[i:i+wind].idxmax())

slmin, intercmin = np.polyfit(minid, minim, 1)
slmax, intercmax = np.polyfit(maxid, maxim, 1)

dfpl = df[candleid-backcandles:candleid+backcandles]
fig = go.Figure(data = [go.Candlestick(x=dfpl.index,
                                       open = dfpl['open'],
                                       high = dfpl['high'],
                                       low = dfpl['low'],
                                       close = dfpl['close'])])
adjintercmin = (df.low.iloc[minid] - slmin*minid).min()
adjintercmax = (df.high.iloc[maxid] - slmax*maxid).max()
fig.add_trace(go.Scatter(x=minid, y=slmin*minid+adjintercmin, mode='lines', name='minimum slope'))
fig.add_trace(go.Scatter(x=maxid, y=slmax*maxid+adjintercmax, mode='lines', name='maximum slope'))

In [ ]:
backcandles = 100 #6 * 30
brange = 70 #backcandles//5
wind = 5
candleid = 4800

optbackcandles = backcandles
sldiff = 100
sldist = 1000

for r1 in range(backcandles-brange, backcandles+brange):
  maxim = np.array([])
  minim = np.array([])
  minid = np.array([])
  maxid = np.array([])
  for i in range(candleid - r1, candleid + 1, wind):
    minim = np.append(minim, df.low.iloc[i:i+wind].min())
    minid = np.append(minid, df.low.iloc[i:i+wind].idxmin())
  for i in range(candleid - r1, candleid + 1, wind):
    maxim = np.append(maxim, df.low.iloc[i:i+wind].max())
    maxid = np.append(maxid, df.low.iloc[i:i+wind].idxmax())
  slmin, intercmin = np.polyfit(minid, minim, 1)
  slmax, intercmax = np.polyfit(maxid, maxim, 1)
  dist = (slmax*candleid+intercmax)-(slmin*candleid-intercmin)
  if(dist<sldist):
    sldiff = abs(slmin-slmax)
    sldist = dist
    slmaxopt = slmax
    slminopt = slmin
    optbackcandles = r1
    intercminopt = intercmin
    intercmaxopt = intercmax
    maximopt = maxim.copy()
    minimopt = minim.copy()
    maxidopt = maxid.copy()
    minidopt = minid.copy()

print(optbackcandles)
dfpl = df[candleid-wind-optbackcandles-backcandles:candleid+optbackcandles]
fig = go.Figure(data = [go.Candlestick(x=dfpl.index,
                                       open = dfpl['open'],
                                       high = dfpl['high'],
                                       low = dfpl['low'],
                                       close = dfpl['close'])])
adjintercmin = (df.low.iloc[minidopt] - slminopt*minidopt).min()
adjintercmax = (df.high.iloc[maxidopt] - slmaxopt*maxidopt).max()
fig.add_trace(go.Scatter(x=minidopt, y=slminopt*minidopt+adjintercmin, mode='lines', name='minimum slope'))
fig.add_trace(go.Scatter(x=maxidopt, y=slmaxopt*maxidopt+adjintercmax, mode='lines', name='maximum slope'))

169


In [ ]:
# backcandles = 100
# wind = 5
# candleid = 4890

# maxim = np.array([])
# minim = np.array([])
# minid = np.array([])
# maxid = np.array([])

# for i in range(candleid - backcandles, candleid + 1, wind):
#   minim = np.append(minim, df.low.iloc[i:i+wind].min())
#   minid = np.append(minid, df.low.iloc[i:i+wind].idxmin())
# for i in range(candleid - backcandles, candleid + 1, wind):
#   maxim = np.append(maxim, df.low.iloc[i:i+wind].max())
#   maxid = np.append(maxid, df.low.iloc[i:i+wind].idxmax())

# slmin, intercmin = np.polyfit(minid, minim, 1)
# slmax, intercmax = np.polyfit(maxid, maxim, 1)

# dfpl = df[candleid-backcandles:candleid+backcandles]
# fig = go.Figure(data = [go.Candlestick(x=dfpl.index,
#                                        open = dfpl['open'],
#                                        high = dfpl['high'],
#                                        low = dfpl['low'],
#                                        close = dfpl['close'])])
# fig.add_trace(go.Scatter(x=minid, y=slmin*minid+intercmin, mode='lines', name='minimum slope'))
# fig.add_trace(go.Scatter(x=maxid, y=slmax*maxid+intercmax, mode='lines', name='maximum slope'))

In [ ]:
# adjintercmin = df.low.loc[candleid-backcandles:candleid].min() - slmin*df.low.iloc[candleid-backcandles:candleid].idxmin()
# adjintercmax = df.high.loc[candleid-backcandles:candleid].max() - slmax*df.high.iloc[candleid-backcandles:candleid].idxmax()

# fig.add_trace(go.Scatter(x=minid, y=slmin*minid+adjintercmin, mode='lines', name='minimum slope'))
# fig.add_trace(go.Scatter(x=maxid, y=slmax*maxid+adjintercmax, mode='lines', name='maximum slope'))
# fig.show()